In [1]:
import numpy as np
def g(x):
    a, b = 1, 100
    dx = -2 * (a - x[0]) - 4 * b * x[0] * (x[1] - x[0] ** 2)
    dy = 2 * b * (x[1] - x[0] ** 2)
    return np.array([dx, dy])


def projection(x, X):
    # Ising the whole R2 space
    return x


def MSGP(x0, X, r=0.01, rho=0.9, sigma=0.0001, epsilon=1e-6, beta=0.5, delta=0.1, max_iter=1000000):
    xk = x0
    n = len(xk)
    sk = np.zeros(n)
    yk = np.zeros(n)

    for k in range(max_iter):
        gk = g(xk)
        if np.linalg.norm(gk) < epsilon:
            print(f"Convergence achieved in {k} iterations")
            return xk

        if k == 0:
            dk = -gk
        else:
            lambda_k = np.zeros(n)
            for i in range(n):
                if sk[i] != 0:
                    if yk[i] / sk[i] > 0:
                        lambda_k[i] = yk[i] / sk[i] # Modified Barzilai-Borwein method
                    else:
                        denom = np.dot(sk, sk)
                        if denom != 0:
                            lambda_k[i] = np.dot(sk, yk) / denom
                lambda_k[i] = max(min(lambda_k[i], 1 / epsilon), epsilon)
                lambda_k[i] = np.where(lambda_k[i] <= epsilon or lambda_k[i] >= 1 / epsilon, delta, lambda_k[i])

            Dk = np.diag(1 / lambda_k)
            dk = -np.dot(Dk, gk)

        # Line search
        alpha_k = beta
        while True:
            new_xk = xk + alpha_k * dk
            if -np.dot(g(new_xk), dk) >= sigma * alpha_k * np.linalg.norm(g(new_xk)) * np.linalg.norm(dk):
                break
            alpha_k *= rho

        xk = xk + alpha_k * dk

        if np.linalg.norm(xk - (xk - alpha_k * dk)) < epsilon:
            print(f"Convergence achieved in {k + 1} iterations")
            return xk

        sk = xk - (xk - alpha_k * dk)
        yk = g(xk) - g(xk - alpha_k * dk)

    print("Maximum iterations reached without convergence.")
    return xk

X = None  # No bounds
x0 = np.array([-1.2, 1.0])  # Initial guess
result = MSGP(x0, X)
print("Solution:", result)


Convergence achieved in 10762 iterations
Solution: [1.00092658 1.00185647]
